### Combining NAAF-corrected *de novo* and PeaksDB  peptides for Trocas 7 incubation samples

The dataset:

    16 treatments: 4 stations, 2 timepoints (Time 0 an Time 24 hrs), 2 size fractions (GFF; GF75):
    
    Stations: 
    
        - Macapa South (MS) South stem, upriver
        - Macapa North (MN) North stem, upriver
        - Chaves (CV) South stem, downriver
        - Baylique (BY) North stem, downriver


    Proteomics samples from 2 trips to UWPR (July 2020 on the QE; April 2021 on the Fusion)
    There were at least triplicate samples for each treatment
    Many in April 2021 injected twice

Starting with:

    Peaks de novo results of PTM-optimized sequencing, NAAF corrected per sample
    PeaksDB de novo-assisted results from PTM-optimized database searches, NAAF corrected per sample
    
    Multiple samples per treatment

Goal:

    CSVs with combined de novo and PeaksDB for each sample, normalized to by NAAF
    
Using:

    - pandas
    - matplotlib
    - numpy

In [5]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [6]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations


### 1. Macapa North, timepoint 0, size fraction 0.3-0.7 um (denoted as MN_T00_GF)
### T7 samples #s: 121, 126
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/126_MN_T00_GF_PDB
    Trocas-incubations/processed/PeaksDB/126_MN_T00_GF_DN

In [8]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_126a = pd.read_csv("processed/PeaksDN/126_MN_T00_GF_DN/126_MN_T00_GF_DN50_naaf_totals.csv")
peaks50_126b = pd.read_csv("processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_naaf_totals.csv")
peaksdb_126a = pd.read_csv("processed/PeaksDB/126_MN_T00_GF_PDB/126_MN_T00_GF_PDB_naaf_totals.csv")
peaksdb_126b = pd.read_csv("processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_naaf_totals.csv")

frames = [peaks50_126a, peaks50_126b, peaksdb_126a, peaksdb_126b]
index = ['peaks50_126a', 'peaks50_126b', 'peaksdb_126a', 'peaksdb_126b']

# concatenate dataframes
tot_126 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_126 = tot_126.loc[:, ~tot_126.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_126', 'peaks50_121', 'peaksdb_126', 'peaksdb_121']
tot_126.insert(loc=0, column='data source', value=names)
tot_126.set_index('data source')

# sum the AAs and PTMN

index = ['126']

data = {'NAAF': tot_126['NAAF'].sum(),
        'A-NAAF': tot_126['A'].sum(),
        'C-NAAF': tot_126['C'].sum(),
        'D-NAAF': tot_126['D'].sum(),
        'E-NAAF': tot_126['E'].sum(),
        'F-NAAF': tot_126['F'].sum(),
        'G-NAAF': tot_126['G'].sum(),
        'H-NAAF': tot_126['H'].sum(),
        'I-NAAF': tot_126['I'].sum(),
        'K-NAAF': tot_126['K'].sum(),
        'L-NAAF': tot_126['L'].sum(),
        'M-NAAF': tot_126['M'].sum(),
        'N-NAAF': tot_126['N'].sum(),
        'P-NAAF': tot_126['P'].sum(),
        'Q-NAAF': tot_126['Q'].sum(),
        'R-NAAF': tot_126['R'].sum(),
        'S-NAAF': tot_126['S'].sum(),
        'T-NAAF': tot_126['T'].sum(),
        'V-NAAF': tot_126['V'].sum(),
        'W-NAAF': tot_126['W'].sum(),
        'Y-NAAF': tot_126['Y'].sum(),
        'C-carb-NAAF': tot_126['c-carb'].sum(),
        'M-oxid-NAAF': tot_126['m-oxid'].sum(),
        'N-deam-NAAF': tot_126['n-deam'].sum(),
        'Q-deam-NAAF': tot_126['q-deam'].sum()
       }

sum_126 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_126['AA-NAAF'] = sum_126['A-NAAF'] + sum_126['C-NAAF'] + sum_126['D-NAAF'] + sum_126['E-NAAF'] + \
                     sum_126['F-NAAF'] + sum_126['G-NAAF'] + sum_126['H-NAAF'] + sum_126['I-NAAF'] + \
                     sum_126['K-NAAF'] + sum_126['L-NAAF'] + sum_126['M-NAAF'] + sum_126['N-NAAF'] + \
                     sum_126['P-NAAF'] + sum_126['Q-NAAF'] + sum_126['R-NAAF'] + sum_126['S-NAAF'] + \
                     sum_126['T-NAAF'] + sum_126['V-NAAF'] + sum_126['W-NAAF'] + sum_126['Y-NAAF'] 

sum_126['AA-modifiable-NAAF'] = sum_126['M-NAAF'] + sum_126['N-NAAF'] + sum_126['Q-NAAF']
                                

sum_126['Cys. w/ carb.'] = sum_126['C-carb-NAAF'] / sum_126['C-NAAF'] #1
sum_126['Met. w/ oxid.'] = sum_126['M-oxid-NAAF'] / sum_126['M-NAAF'] #2
sum_126['Asn. w/ deam.'] = sum_126['N-deam-NAAF'] / sum_126['N-NAAF'] #3
sum_126['Glu. w/ deam.'] = sum_126['Q-deam-NAAF'] / sum_126['Q-NAAF'] #3


sum_126['Overall modified'] = 0.001*((sum_126['Met. w/ oxid.']*sum_126['M-NAAF']) + \
                                     (sum_126['Asn. w/ deam.']*sum_126['N-NAAF']) + \
                                     (sum_126['Glu. w/ deam.']*sum_126['Q-NAAF']) / (sum_126['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_126.to_csv("processed/NAAF-sums/NAAF-sum-126.csv")

sum_126.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
126,636.722578,557.025632,5.666585,24.194401,32.845163,8.466211,68.467798,14.883078,28.861269,27.870881,...,1.105145,2.886811,18.844915,5531.593219,207.617108,1.0,0.07239,0.018988,0.467456,0.004083


### 2. Macapa North, timepoint 0, size fraction>0.7 um (denoted as MN_T00_GD)
### T7 samples #s: 221, 226
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/226_MN_T00_GD_PDB
    Trocas-incubations/processed/PeaksDB/226_MN_T00_GD_DN

In [9]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_226a = pd.read_csv("processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_naaf_totals.csv")
peaks50_226b = pd.read_csv("processed/PeaksDN/226_MN_T00_GD_DN/226B_MN_T00_GD_DN50_naaf_totals.csv")
peaksdb_226a = pd.read_csv("processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_naaf_totals.csv")
peaksdb_226b = pd.read_csv("processed/PeaksDB/226_MN_T00_GD_PDB/226B_MN_T00_GD_PDB_naaf_totals.csv")

frames = [peaks50_226a, peaks50_226b, peaksdb_226a, peaksdb_226b]
index = ['peaks50_226a', 'peaks50_226b', 'peaksdb_226a', 'peaksdb_226b']

# concatenate dataframes
tot_226 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_226 = tot_226.loc[:, ~tot_226.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_226', 'peaks50_209', 'peaksdb_226', 'peaksdb_209']
tot_226.insert(loc=0, column='data source', value=names)
tot_226.set_index('data source')

# sum the AAs and PTMN

index = ['226']

data = {'NAAF': tot_226['NAAF'].sum(),
        'A-NAAF': tot_226['A'].sum(),
        'C-NAAF': tot_226['C'].sum(),
        'D-NAAF': tot_226['D'].sum(),
        'E-NAAF': tot_226['E'].sum(),
        'F-NAAF': tot_226['F'].sum(),
        'G-NAAF': tot_226['G'].sum(),
        'H-NAAF': tot_226['H'].sum(),
        'I-NAAF': tot_226['I'].sum(),
        'K-NAAF': tot_226['K'].sum(),
        'L-NAAF': tot_226['L'].sum(),
        'M-NAAF': tot_226['M'].sum(),
        'N-NAAF': tot_226['N'].sum(),
        'P-NAAF': tot_226['P'].sum(),
        'Q-NAAF': tot_226['Q'].sum(),
        'R-NAAF': tot_226['R'].sum(),
        'S-NAAF': tot_226['S'].sum(),
        'T-NAAF': tot_226['T'].sum(),
        'V-NAAF': tot_226['V'].sum(),
        'W-NAAF': tot_226['W'].sum(),
        'Y-NAAF': tot_226['Y'].sum(),
        'C-carb-NAAF': tot_226['c-carb'].sum(),
        'M-oxid-NAAF': tot_226['m-oxid'].sum(),
        'N-deam-NAAF': tot_226['n-deam'].sum(),
        'Q-deam-NAAF': tot_226['q-deam'].sum()
       }

sum_226 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_226['AA-NAAF'] = sum_226['A-NAAF'] + sum_226['C-NAAF'] + sum_226['D-NAAF'] + sum_226['E-NAAF'] + \
                     sum_226['F-NAAF'] + sum_226['G-NAAF'] + sum_226['H-NAAF'] + sum_226['I-NAAF'] + \
                     sum_226['K-NAAF'] + sum_226['L-NAAF'] + sum_226['M-NAAF'] + sum_226['N-NAAF'] + \
                     sum_226['P-NAAF'] + sum_226['Q-NAAF'] + sum_226['R-NAAF'] + sum_226['S-NAAF'] + \
                     sum_226['T-NAAF'] + sum_226['V-NAAF'] + sum_226['W-NAAF'] + sum_226['Y-NAAF'] 

sum_226['AA-modifiable-NAAF'] = sum_226['M-NAAF'] + sum_226['N-NAAF'] + sum_226['Q-NAAF']
                                

sum_226['Cys. w/ carb.'] = sum_226['C-carb-NAAF'] / sum_226['C-NAAF'] #1
sum_226['Met. w/ oxid.'] = sum_226['M-oxid-NAAF'] / sum_226['M-NAAF'] #2
sum_226['Asn. w/ deam.'] = sum_226['N-deam-NAAF'] / sum_226['N-NAAF'] #3
sum_226['Glu. w/ deam.'] = sum_226['Q-deam-NAAF'] / sum_226['Q-NAAF'] #3


sum_226['Overall modified'] = 0.001*((sum_226['Met. w/ oxid.']*sum_226['M-NAAF']) + \
                                     (sum_226['Asn. w/ deam.']*sum_226['N-NAAF']) + \
                                     (sum_226['Glu. w/ deam.']*sum_226['Q-NAAF']) / (sum_226['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_226.to_csv("processed/NAAF-sums/NAAF-sum-226.csv")

sum_226.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
226,493.171494,304.409541,27.938303,19.470353,39.861193,28.143419,63.365831,47.493075,59.908234,46.725263,...,14.187238,7.214924,1.259725,4211.087397,124.117883,1.0,0.353521,0.10187,0.095711,0.021412


### 3. Macapa North, timepoint 24 hrs, size fraction 0.3-0.7 um (denoted as MN_T24_GF)
### T7 samples #s: 318A, 318B, 319 (318C)
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/318_MN_T24_GF_PDB
    Trocas-incubations/processed/PeaksDB/318_MN_T24_GF_DN

In [10]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_318a = pd.read_csv("processed/PeaksDN/318_MN_T24_GF_DN/318A_MN_T24_GF_DN50_naaf_totals.csv")
peaks50_318b = pd.read_csv("processed/PeaksDN/318_MN_T24_GF_DN/318B_MN_T24_GF_DN50_naaf_totals.csv")
peaks50_318c = pd.read_csv("processed/PeaksDN/318_MN_T24_GF_DN/318C_MN_T24_GF_DN50_naaf_totals.csv")
peaksdb_318a = pd.read_csv("processed/PeaksDB/318_MN_T24_GF_PDB/318A_MN_T24_GF_PDB_naaf_totals.csv")
peaksdb_318b = pd.read_csv("processed/PeaksDB/318_MN_T24_GF_PDB/318B_MN_T24_GF_PDB_naaf_totals.csv")
peaksdb_318c = pd.read_csv("processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_naaf_totals.csv")

frames = [peaks50_318a, peaks50_318b, peaks50_318c, peaksdb_318a, peaksdb_318b, peaksdb_318c]
index = ['peaks50_318a', 'peaks50_318b', 'peaks50_318c', 'peaksdb_318a', 'peaksdb_318b', 'peaksdb_318bc']

# concatenate dataframes
tot_318 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_318 = tot_318.loc[:, ~tot_318.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_318a', 'peaks50_318b', 'peaks50_319', 'peaksdb_318a', 'peaksdb_318b', 'peaksdb_319']
tot_318.insert(loc=0, column='data source', value=names)
tot_318.set_index('data source')

# sum the AAs and PTMN

index = ['318']

data = {'NAAF': tot_318['NAAF'].sum(),
        'A-NAAF': tot_318['A'].sum(),
        'C-NAAF': tot_318['C'].sum(),
        'D-NAAF': tot_318['D'].sum(),
        'E-NAAF': tot_318['E'].sum(),
        'F-NAAF': tot_318['F'].sum(),
        'G-NAAF': tot_318['G'].sum(),
        'H-NAAF': tot_318['H'].sum(),
        'I-NAAF': tot_318['I'].sum(),
        'K-NAAF': tot_318['K'].sum(),
        'L-NAAF': tot_318['L'].sum(),
        'M-NAAF': tot_318['M'].sum(),
        'N-NAAF': tot_318['N'].sum(),
        'P-NAAF': tot_318['P'].sum(),
        'Q-NAAF': tot_318['Q'].sum(),
        'R-NAAF': tot_318['R'].sum(),
        'S-NAAF': tot_318['S'].sum(),
        'T-NAAF': tot_318['T'].sum(),
        'V-NAAF': tot_318['V'].sum(),
        'W-NAAF': tot_318['W'].sum(),
        'Y-NAAF': tot_318['Y'].sum(),
        'C-carb-NAAF': tot_318['c-carb'].sum(),
        'M-oxid-NAAF': tot_318['m-oxid'].sum(),
        'N-deam-NAAF': tot_318['n-deam'].sum(),
        'Q-deam-NAAF': tot_318['q-deam'].sum()
       }

sum_318 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_318['AA-NAAF'] = sum_318['A-NAAF'] + sum_318['C-NAAF'] + sum_318['D-NAAF'] + sum_318['E-NAAF'] + \
                     sum_318['F-NAAF'] + sum_318['G-NAAF'] + sum_318['H-NAAF'] + sum_318['I-NAAF'] + \
                     sum_318['K-NAAF'] + sum_318['L-NAAF'] + sum_318['M-NAAF'] + sum_318['N-NAAF'] + \
                     sum_318['P-NAAF'] + sum_318['Q-NAAF'] + sum_318['R-NAAF'] + sum_318['S-NAAF'] + \
                     sum_318['T-NAAF'] + sum_318['V-NAAF'] + sum_318['W-NAAF'] + sum_318['Y-NAAF'] 

sum_318['AA-modifiable-NAAF'] = sum_318['M-NAAF'] + sum_318['N-NAAF'] + sum_318['Q-NAAF']
                                

sum_318['Cys. w/ carb.'] = sum_318['C-carb-NAAF'] / sum_318['C-NAAF'] #1
sum_318['Met. w/ oxid.'] = sum_318['M-oxid-NAAF'] / sum_318['M-NAAF'] #2
sum_318['Asn. w/ deam.'] = sum_318['N-deam-NAAF'] / sum_318['N-NAAF'] #3
sum_318['Glu. w/ deam.'] = sum_318['Q-deam-NAAF'] / sum_318['Q-NAAF'] #3


sum_318['Overall modified'] = 0.001*((sum_318['Met. w/ oxid.']*sum_318['M-NAAF']) + \
                                     (sum_318['Asn. w/ deam.']*sum_318['N-NAAF']) + \
                                     (sum_318['Glu. w/ deam.']*sum_318['Q-NAAF']) / (sum_318['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_318.to_csv("processed/NAAF-sums/NAAF-sum-318.csv")

sum_318.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
318,1061.598087,901.920654,236.868468,42.715742,103.033273,101.90403,335.807287,135.955659,101.49832,216.653986,...,50.124589,32.273241,4.366063,10146.029012,484.468834,1.0,0.387675,0.127555,0.042738,0.082407


### 4. Macapa North, timepoint 24 hrs, size fraction >0.7 um (denoted as MN_T24_GD)
### T7 samples #s: 417A, 417B
### Exported NAAF and stripped peptides contained in the following directories:

    Trocas-incubations/processed/PeaksDB/417_MN_T24_GD_PDB
    Trocas-incubations/processed/PeaksDB/417_MN_T24_GD_DN

In [11]:
# for each of the 4 incubation samples:
# read in NAAF totals csvs made in PeaksDN (de novo), and PeaksDB (de novo assisted database search) notebooks
# bringing de novo peptides >50% ALC and PeaksDB peptides <1% FDR

peaks50_417a = pd.read_csv("processed/PeaksDN/417_MN_T24_GD_DN/417A_MN_T24_GD_DN50_naaf_totals.csv")
peaks50_417b = pd.read_csv("processed/PeaksDN/417_MN_T24_GD_DN/417B_MN_T24_GD_DN50_naaf_totals.csv")
peaksdb_417a = pd.read_csv("processed/PeaksDB/417_MN_T24_GD_PDB/417A_MN_T24_GD_PDB_naaf_totals.csv")
peaksdb_417b = pd.read_csv("processed/PeaksDB/417_MN_T24_GD_PDB/417B_MN_T24_GD_PDB_naaf_totals.csv")


frames = [peaks50_417a, peaks50_417b, peaksdb_417a, peaksdb_417b, ]
index = ['peaks50_417a', 'peaks50_417b', 'peaksdb_417a', 'peaksdb_417b']

# concatenate dataframes
tot_417 = pd.concat(frames, sort=False, ignore_index=True)

# there's a useless column in there
tot_417 = tot_417.loc[:, ~tot_417.columns.str.contains('^Unnamed')]

# reindex with peaks 
names = ['peaks50_417a', 'peaks50_417b', 'peaksdb_417a', 'peaksdb_417b']
tot_417.insert(loc=0, column='data source', value=names)
tot_417.set_index('data source')

# sum the AAs and PTMN

index = ['417']

data = {'NAAF': tot_417['NAAF'].sum(),
        'A-NAAF': tot_417['A'].sum(),
        'C-NAAF': tot_417['C'].sum(),
        'D-NAAF': tot_417['D'].sum(),
        'E-NAAF': tot_417['E'].sum(),
        'F-NAAF': tot_417['F'].sum(),
        'G-NAAF': tot_417['G'].sum(),
        'H-NAAF': tot_417['H'].sum(),
        'I-NAAF': tot_417['I'].sum(),
        'K-NAAF': tot_417['K'].sum(),
        'L-NAAF': tot_417['L'].sum(),
        'M-NAAF': tot_417['M'].sum(),
        'N-NAAF': tot_417['N'].sum(),
        'P-NAAF': tot_417['P'].sum(),
        'Q-NAAF': tot_417['Q'].sum(),
        'R-NAAF': tot_417['R'].sum(),
        'S-NAAF': tot_417['S'].sum(),
        'T-NAAF': tot_417['T'].sum(),
        'V-NAAF': tot_417['V'].sum(),
        'W-NAAF': tot_417['W'].sum(),
        'Y-NAAF': tot_417['Y'].sum(),
        'C-carb-NAAF': tot_417['c-carb'].sum(),
        'M-oxid-NAAF': tot_417['m-oxid'].sum(),
        'N-deam-NAAF': tot_417['n-deam'].sum(),
        'Q-deam-NAAF': tot_417['q-deam'].sum()
       }

sum_417 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q
# also calculate the NAAF-corrected % of 'total modifiable AA space'
# this is a weighted average of modified residues wrt their relative abundance in the combined peptide set

sum_417['AA-NAAF'] = sum_417['A-NAAF'] + sum_417['C-NAAF'] + sum_417['D-NAAF'] + sum_417['E-NAAF'] + \
                     sum_417['F-NAAF'] + sum_417['G-NAAF'] + sum_417['H-NAAF'] + sum_417['I-NAAF'] + \
                     sum_417['K-NAAF'] + sum_417['L-NAAF'] + sum_417['M-NAAF'] + sum_417['N-NAAF'] + \
                     sum_417['P-NAAF'] + sum_417['Q-NAAF'] + sum_417['R-NAAF'] + sum_417['S-NAAF'] + \
                     sum_417['T-NAAF'] + sum_417['V-NAAF'] + sum_417['W-NAAF'] + sum_417['Y-NAAF'] 

sum_417['AA-modifiable-NAAF'] = sum_417['M-NAAF'] + sum_417['N-NAAF'] + sum_417['Q-NAAF']
                                

sum_417['Cys. w/ carb.'] = sum_417['C-carb-NAAF'] / sum_417['C-NAAF'] #1
sum_417['Met. w/ oxid.'] = sum_417['M-oxid-NAAF'] / sum_417['M-NAAF'] #2
sum_417['Asn. w/ deam.'] = sum_417['N-deam-NAAF'] / sum_417['N-NAAF'] #3
sum_417['Glu. w/ deam.'] = sum_417['Q-deam-NAAF'] / sum_417['Q-NAAF'] #3


sum_417['Overall modified'] = 0.001*((sum_417['Met. w/ oxid.']*sum_417['M-NAAF']) + \
                                     (sum_417['Asn. w/ deam.']*sum_417['N-NAAF']) + \
                                     (sum_417['Glu. w/ deam.']*sum_417['Q-NAAF']) / (sum_417['AA-modifiable-NAAF'])) 
                                                            

# write to a csv T7 combined data directory in /processed
sum_417.to_csv("processed/NAAF-sums/NAAF-sum-417.csv")

sum_417.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,M-oxid-NAAF,N-deam-NAAF,Q-deam-NAAF,AA-NAAF,AA-modifiable-NAAF,Cys. w/ carb.,Met. w/ oxid.,Asn. w/ deam.,Glu. w/ deam.,Overall modified
417,729.271349,520.612943,17.220006,24.356419,23.603088,17.978779,51.999835,11.478567,25.248183,33.329434,...,9.696427,6.529224,19.057003,6129.595573,193.08416,1.0,0.591274,0.04458,0.630554,0.016324
